Meeting notes:

##### Make diag.bib update code (notebook)
- Compare titles of bib and semantic scholar
- Match
- If match:
    - Add semantic scholar ID to diag bib
    - Update gscites field from bib entry
- If no match:
    - Check best matches, if DOI and Arxiv --> chose DOI 
    - Define right fields (look at code James) + citations + semantic scolar ID 
    - Add 'automatically added' field so we can trace back automation errors more easilty
    - Add to bib   
    
##### Make sript that checks ss ids and adds new ones to diag bib
    
    
##### Make script that can be used by DIAG members
- Add doi/arxiv
- if DOI check if Arxiv exists, if so --> remove Arxiv
- Sort right fields from DOI --> add fields
- Search for semantic scholar ID
- Print fields so user can check it
- Add to bib
- Manual commit push

##### Make script that checks all semantic scholar IDs and updates gscites
- github action overnight?

##### Note: 
- requests limit, 100 requests per 5 mins for ss 
    - 100 papers at once = 1 request
- google doc: 
    - https://docs.google.com/document/d/1uFyh5Z-wieksq_4Bf8Br7wHpN-buHECunS0VurYnlvg/edit


# Libraries

In [2]:
import os
import numpy as np
import pandas as pd

import requests
# from bibreader import parse_bibtex_file, get_bib_blocks
from bib_handling_code.processbib import read_bibfile
from bib_handling_code.processbib import save_to_file
from difflib import SequenceMatcher


# Example from url giving 100 papers from one of Bram's semantic scolar IDs

In [3]:
staff_id = 8038506
staff_ids = [8038506]

In [4]:
url=f'https://api.semanticscholar.org/graph/v1/author/{staff_id}/papers?fields=year,title,authors,externalIds,citationCount&limit=100'

r = requests.get(url)
ss_staff_data = r.json()['data']

In [21]:
ss_staff_data[0]

{'paperId': '1898998f27ce0750a42f8f3ca2ed7e292f05aee2',
 'externalIds': {'PubMedCentral': '10029046',
  'DBLP': 'journals/corr/abs-2302-01790',
  'ArXiv': '2302.01790',
  'DOI': '10.48550/arXiv.2302.01790',
  'CorpusId': 256598062,
  'PubMed': '36945687'},
 'title': 'Understanding metric-related pitfalls in image analysis validation',
 'year': 2023,
 'citationCount': 3,
 'authors': [{'authorId': '47131776', 'name': 'Annika Reinke'},
  {'authorId': '14332753', 'name': 'M. Tizabi'},
  {'authorId': '2065793436', 'name': 'M. Baumgartner'},
  {'authorId': '33751657', 'name': 'M. Eisenmann'},
  {'authorId': '2072738748', 'name': 'Doreen Heckmann-Notzel'},
  {'authorId': '2204468751', 'name': 'A. E. Kavu'},
  {'authorId': '2072737475', 'name': 'Tim Radsch'},
  {'authorId': '1697589', 'name': 'C. Sudre'},
  {'authorId': '3748566', 'name': 'L. Ación'},
  {'authorId': '51504413', 'name': 'M. Antonelli'},
  {'authorId': '1699104', 'name': 'T. Arbel'},
  {'authorId': '3199900', 'name': 'S. Bakas'}

# Function to get data from DOI/ArXiv

In [12]:
def get_doi_csl(doi):
    response = requests.get(
        f"https://doi.org/{doi}",
        headers={"Accept": "application/vnd.citationstyles.csl+json"},
        timeout=5,
    )
    response.raise_for_status()
    return response.json()

# Example

In [13]:
'arXiv' in '10.48550/arXiv.2302.01738' 

True

In [14]:
get_doi_csl('10.48550/arXiv.2302.01738').keys()

dict_keys(['type', 'id', 'categories', 'author', 'issued', 'abstract', 'DOI', 'publisher', 'title', 'URL', 'copyright', 'version'])

In [15]:
ss_staff_data[2]['externalIds']

{'ArXiv': '2302.01738',
 'DBLP': 'journals/corr/abs-2302-01738',
 'DOI': '10.48550/arXiv.2302.01738',
 'CorpusId': 256598301}

In [16]:
found_arxiv_doi = False
found_regular_doi = False

In [17]:
#ArXiv
for ss_staff_entry in ss_staff_data:
    if 'DOI' in ss_staff_entry['externalIds']:
        if 'arXiv' in ss_staff_entry['externalIds']['DOI']:
            arxiv_doi = ss_staff_entry['externalIds']['DOI']
            found_arxiv_doi = True
    if found_arxiv_doi:
        break
#DOI
for ss_staff_entry in ss_staff_data:
    if 'DOI' in ss_staff_entry['externalIds']:
        if 'arXiv' not in ss_staff_entry['externalIds']['DOI']:
            regular_doi = ss_staff_entry['externalIds']['DOI']
            found_regular_doi = True
    if found_regular_doi:
        break

print(arxiv_doi)
print(regular_doi)

10.48550/arXiv.2302.01790
10.1007/978-3-031-28241-6_18


In [18]:
doi = regular_doi

In [19]:
doi_data = get_doi_csl(doi)
doi_data

{'indexed': {'date-parts': [[2023, 3, 16]],
  'date-time': '2023-03-16T04:50:23Z',
  'timestamp': 1678942223397},
 'publisher-location': 'Cham',
 'reference-count': 10,
 'publisher': 'Springer Nature Switzerland',
 'isbn-type': [{'value': '9783031282409', 'type': 'print'},
  {'value': '9783031282416', 'type': 'electronic'}],
 'license': [{'start': {'date-parts': [[2023, 1, 1]],
    'date-time': '2023-01-01T00:00:00Z',
    'timestamp': 1672531200000},
   'content-version': 'tdm',
   'delay-in-days': 0,
   'URL': 'https://www.springernature.com/gp/researchers/text-and-data-mining'},
  {'start': {'date-parts': [[2023, 1, 1]],
    'date-time': '2023-01-01T00:00:00Z',
    'timestamp': 1672531200000},
   'content-version': 'vor',
   'delay-in-days': 0,
   'URL': 'https://www.springernature.com/gp/researchers/text-and-data-mining'}],
 'content-domain': {'domain': ['link.springer.com'],
  'crossmark-restriction': False},
 'published-print': {'date-parts': [[2023]]},
 'DOI': '10.1007/978-3-031-

In [20]:
[print(key) for key in doi_data.keys()];

indexed
publisher-location
reference-count
publisher
isbn-type
license
content-domain
published-print
DOI
type
created
page
update-policy
source
is-referenced-by-count
title
prefix
author
member
published-online
reference
container-title
original-title
link
deposited
score
resource
subtitle
short-title
issued
ISBN
references-count
URL
relation
ISSN
published
assertion


# ####################################
#  FINAL
# ####################################

In [1]:
# df_bib_data = pd.read_csv('script_data/temp_diag_bib_summary.csv')
# df_all_staff_id_ss_data = pd.read_csv('script_data/temp_semantic_scholar_all_staff_id_last_100_summary.csv')
# df_doi_match_data = pd.read_csv('script_data/temp_doi_match_data.csv')
# df_title_match_data = pd.read_csv('script_data/temp_title_match_data.csv')

In [7]:
df_bib_data

,bibkey,title,doi,gs_citations
0,AA,NaN,NaN,NaN
1,AAC,NaN,NaN,NaN
2,AACC,NaN,NaN,NaN
3,AAPM,NaN,NaN,NaN
4,AATC,NaN,NaN,NaN
...,...,...,...,...
2778,Sadr23,Deep Learning for Detection of Periapical Radi...,10.1016/j.joen.2022.12.007,NaN
2779,Anto23,Retrospective validation of nodule management ...,NaN,NaN
2780,Sido23,Multi-source data approach for personalized ou...,10.1007/s10654-023-00975-9,NaN
2781,Graa23,MRI image features with an evident relation to...,10.1007/s00586-023-07602-x,NaN


In [8]:
df_all_staff_id_ss_data

,staff_id,ss_id,title,doi,ss_citations
0,8038506,1898998f27ce0750a42f8f3ca2ed7e292f05aee2,Understanding metric-related pitfalls in image...,10.48550/arXiv.2302.01790,3
1,8038506,362c510dec0d566d22d5be3af0519fc7eec8bb86,Uncertainty-Aware Multiple-Instance Learning f...,NaN,0
2,8038506,7981606bf8110ec6cc64baa22d694096f7862939,AIROGS: Artificial Intelligence for RObust Gla...,10.48550/arXiv.2302.01738,2
3,8038506,e738d94746b0f33b1c48737022ab186618d4d4cd,SimpleRad: Patient-Friendly Dutch Radiology Re...,10.1007/978-3-031-28241-6_18,0
4,8038506,2d49fb82fd715a213e2149f9f08dd2fb4749b235,Challenges in digital medicine applications in...,10.1038/s41467-022-30728-3,3
...,...,...,...,...,...
1103,5752941,7971250a0b35dd47790d09e0c1d1fdcd16c5e802,Future directions in resuscitation care: Towar...,10.1016/J.RESUSCITATION.2018.07.350,0
1104,5752941,3cfe71ef252b69aeaa7d5e70b84afb518e253e82,Ventricular fibrillation waveform characterist...,10.1016/j.resuscitation.2017.03.029,8
1105,5752941,183eca01f54b6aca310e7d68a48a1bd6724745dd,Abstract 19221: Inter Shock Changes of the Ven...,NaN,0
1106,5752941,1b1b6d1c122164f90cecfa7150becf536a99b92d,Abstract 19090: Ventricular Fibrillation Wavef...,NaN,0


In [9]:
df_doi_match_data

,bibkey,ss_id,comment
0,Altu20,90a870cbb9897124193ba18c5358fe45b6260621,"multiple doi matches, took the first one with ..."
1,Amga20,b4c4c3dc91d42114023b0575c3e2273b87446ff7,"multiple doi matches, took the first one with ..."
2,Anto22,979a9f247700d00ff2c3f0612d5eb001379f93c8,"multiple doi matches, took the first one with ..."
3,Apru19,0c44949334771d4e331d792bc0db77c7fe708ca2,"multiple doi matches, took the first one with ..."
4,Ares19,b3dc561dd990cebc626e10318b8582a198aa3571,"multiple doi matches, took the first one with ..."
...,...,...,...
318,Muna21a,79c9209f3892a5a704abb6be161ca7b41d02775d,NaN
319,Hude20,c52daf1cb971120c4083116cfc213acbaac6faaf,"multiple doi matches, took the first one with ..."
320,Vina22,c52daf1cb971120c4083116cfc213acbaac6faaf,NaN
321,Adam22a,516b2dc6c3761458c8fa6f5759295673e86a42db,"multiple doi matches, took the first one with ..."


In [10]:
df_title_match_data

,ss_id,ss_title,max_bibkey,max_bib_title,max_ratio,up80_bibkeys,up80_bib_titles,up80_ratios
0,1898998f27ce0750a42f8f3ca2ed7e292f05aee2,Understanding metric-related pitfalls in image...,Litj17,A Survey on Deep Learning in Medical Image Ana...,0.529915,NaN,NaN,NaN
1,362c510dec0d566d22d5be3af0519fc7eec8bb86,Uncertainty-Aware Multiple-Instance Learning f...,Venh19,Machine Learning for Quantification of Age-Rel...,0.569038,NaN,NaN,NaN
2,7981606bf8110ec6cc64baa22d694096f7862939,AIROGS: Artificial Intelligence for RObust Gla...,Thee20,Artificial intelligence for eye care,0.598131,NaN,NaN,NaN
3,e738d94746b0f33b1c48737022ab186618d4d4cd,SimpleRad: Patient-Friendly Dutch Radiology Re...,Ramo12,Supervised Content Based Image Retrieval Using...,0.556522,NaN,NaN,NaN
4,2d49fb82fd715a213e2149f9f08dd2fb4749b235,Challenges in digital medicine applications in...,Marc22,Empowering digital pathology applications thro...,0.500000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1103,7971250a0b35dd47790d09e0c1d1fdcd16c5e802,Future directions in resuscitation care: Towar...,Sanc10,Improving hard exudate detection in retinal im...,0.447154,NaN,NaN,NaN
1104,3cfe71ef252b69aeaa7d5e70b84afb518e253e82,Ventricular fibrillation waveform characterist...,Hadd20,Characterisation of the tumour-host interface ...,0.442396,NaN,NaN,NaN
1105,183eca01f54b6aca310e7d68a48a1bd6724745dd,Abstract 19221: Inter Shock Changes of the Ven...,Niem06a,Image structure clustering for image quality v...,0.398524,NaN,NaN,NaN
1106,1b1b6d1c122164f90cecfa7150becf536a99b92d,Abstract 19090: Ventricular Fibrillation Wavef...,Amin11,Air trapping on HRCT assessed by quantitative ...,0.379603,NaN,NaN,NaN


# Import diag.bib

In [2]:
path_diag_bib = os.path.join('..', 'diag.bib')
diag_bib_raw = read_bibfile(None, path_diag_bib) # I changed the code in such a way that IF I give a second argument, it uses the second argument as a full path

In [3]:
len(diag_bib_raw)

2783

In [4]:
bib_data = []
for bib_entry in diag_bib_raw:
    bibkey = bib_entry.key
    
    bib_title = None
    bib_doi = None
    bib_gscites = None
    
    if 'title' in bib_entry.fields:
        bib_title = bib_entry.fields['title'].strip('{ }')
    if 'doi' in bib_entry.fields:
        bib_doi = bib_entry.fields['doi'].strip('{ }')
    if 'gscites' in bib_entry.fields:
        bib_gscites = bib_entry.fields['gscites'].strip('{ }')
    
    bib_data.append([bibkey, bib_title, bib_doi, bib_gscites])
bib_columns = ['bibkey', 'title', 'doi', 'gs_citations']
df_bib_data = pd.DataFrame(bib_data, columns=bib_columns)

In [5]:
df_bib_data.to_csv('script_data/temp_diag_bib_summary.csv', index=False)

In [6]:
df_bib_data.tail()

,bibkey,title,doi,gs_citations
2778,Sadr23,Deep Learning for Detection of Periapical Radi...,10.1016/j.joen.2022.12.007,None
2779,Anto23,Retrospective validation of nodule management ...,None,None
2780,Sido23,Multi-source data approach for personalized ou...,10.1007/s10654-023-00975-9,None
2781,Graa23,MRI image features with an evident relation to...,10.1007/s00586-023-07602-x,None
2782,Graa22,Segmentation of vertebrae and intervertebral d...,10.1117/12.2611423,None


Used for title matching

In [8]:
df_bib_data_with_titles = df_bib_data[df_bib_data['title'].notna()]
df_bib_data_with_titles.head()

,bibkey,title,doi,gs_citations
1065,Abas05,Analysis of regularity in the {EEG} background...,10.1016/j.clinph.2005.04.001,233
1066,Genu22,Imaging tumor-infiltrating CD8 (+) T-cells in ...,None,None
1067,Abas05a,EEG} background activity analysis in {A}lzheim...,None,3
1068,Abel19,"Computational pathology definitions, best prac...",10.1002/path.5331,24
1069,Abra08a,Evaluation of a system for automatic detection...,10.2337/dc07-1312,287


# Gather ss and bib data for all staff member IDs

In [9]:
staff_dict = {'Bram van Ginneken': [8038506, 123637526],
'Francesco Ciompi': [143613202],
'Alessa Hering': [153744566],
'Henjan Huisman': [34754023],
'Colin Jacobs': [2895994],
'Peter Koopmans': [34726383],
'Jeroen van der Laak': [145441238, 145388932],
'Geert Litjens': [145959882],
'James Meakin': [4960344],
'Keelin Murphy': [35730362],
'Ajay Patel': [2109170880, 2116215861],
'Cornelia Schaefer-Prokop': [1419819133, 1445069528],
'Matthieu Rutten': [2074975080, 2156546],
'Jos Thannhauser': [5752941]}

In [10]:
staff_ids = []
[staff_ids.extend(values) for values in staff_dict.values()];
staff_ids

[8038506,
 123637526,
 143613202,
 153744566,
 34754023,
 2895994,
 34726383,
 145441238,
 145388932,
 145959882,
 4960344,
 35730362,
 2109170880,
 2116215861,
 1419819133,
 1445069528,
 2074975080,
 2156546,
 5752941]

## Semantic scolar

In [13]:
all_staff_id_ss_data = []

# get data for staff member
for idx_staff_id, staff_id in enumerate(staff_ids):
    print(f'[{idx_staff_id+1}/{len(staff_ids)}]: {staff_id}')
    staff_id_ss_data = []
    # get semantic scolar data
    url=f'https://api.semanticscholar.org/graph/v1/author/{staff_id}/papers?fields=year,title,authors,externalIds,citationCount&limit=100'
    r = requests.get(url)
    ss_staff_data = r.json()['data']
    
    # loop over semantic scolar data entries
    for ss_staff_entry in ss_staff_data:
        ss_id = None
        ss_title = None
        ss_doi = None
        ss_citations = None

        if 'paperId' in ss_staff_entry:
            ss_id = ss_staff_entry['paperId']
        if 'title' in ss_staff_entry:
            ss_title = ss_staff_entry['title']
        if 'DOI' in ss_staff_entry['externalIds']:
            ss_doi = ss_staff_entry['externalIds']['DOI']
        if 'citationCount' in ss_staff_entry:
            ss_citations = ss_staff_entry['citationCount']

        staff_id_ss_data.append([staff_id, ss_id, ss_title, ss_doi, ss_citations])
    all_staff_id_ss_data.extend(staff_id_ss_data)    
ss_columns = ['staff_id', 'ss_id', 'title', 'doi', 'ss_citations']
df_all_staff_id_ss_data = pd.DataFrame(all_staff_id_ss_data, columns=ss_columns)
print('DONE')

[1/19]: 8038506
[2/19]: 123637526
[3/19]: 143613202
[4/19]: 153744566
[5/19]: 34754023
[6/19]: 2895994
[7/19]: 34726383
[8/19]: 145441238
[9/19]: 145388932
[10/19]: 145959882
[11/19]: 4960344
[12/19]: 35730362
[13/19]: 2109170880
[14/19]: 2116215861
[15/19]: 1419819133
[16/19]: 1445069528
[17/19]: 2074975080
[18/19]: 2156546
[19/19]: 5752941
DONE


In [15]:
df_all_staff_id_ss_data

,staff_id,ss_id,title,doi,ss_citations
0,8038506,1898998f27ce0750a42f8f3ca2ed7e292f05aee2,Understanding metric-related pitfalls in image...,10.48550/arXiv.2302.01790,3
1,8038506,362c510dec0d566d22d5be3af0519fc7eec8bb86,Uncertainty-Aware Multiple-Instance Learning f...,None,0
2,8038506,7981606bf8110ec6cc64baa22d694096f7862939,AIROGS: Artificial Intelligence for RObust Gla...,10.48550/arXiv.2302.01738,2
3,8038506,e738d94746b0f33b1c48737022ab186618d4d4cd,SimpleRad: Patient-Friendly Dutch Radiology Re...,10.1007/978-3-031-28241-6_18,0
4,8038506,2d49fb82fd715a213e2149f9f08dd2fb4749b235,Challenges in digital medicine applications in...,10.1038/s41467-022-30728-3,3
...,...,...,...,...,...
1103,5752941,7971250a0b35dd47790d09e0c1d1fdcd16c5e802,Future directions in resuscitation care: Towar...,10.1016/J.RESUSCITATION.2018.07.350,0
1104,5752941,3cfe71ef252b69aeaa7d5e70b84afb518e253e82,Ventricular fibrillation waveform characterist...,10.1016/j.resuscitation.2017.03.029,8
1105,5752941,183eca01f54b6aca310e7d68a48a1bd6724745dd,Abstract 19221: Inter Shock Changes of the Ven...,None,0
1106,5752941,1b1b6d1c122164f90cecfa7150becf536a99b92d,Abstract 19090: Ventricular Fibrillation Wavef...,None,0


In [16]:
df_all_staff_id_ss_data.to_csv('script_data/temp_semantic_scholar_all_staff_id_last_100_summary.csv', index=False)

# Loop for matching DOIs and titles

In [58]:
doi_match_data = []
match_keys_columns = ['bibkey', 'ss_id', 'comment']
print('[MATCHING ON DOIS]')
for idx_bib, row_bib in df_bib_data.iterrows():
    bibkey = row_bib['bibkey']
    comment = None
    
    bib_doi = row_bib['doi']
    if bib_doi == None:
        continue
    doi_matching = df_all_staff_id_ss_data['doi'].isin([bib_doi])
    
    if doi_matching.any():
        matches = df_all_staff_id_ss_data[doi_matching] # could be multiple in SS, search for "Epithelium segmentation using deep learning" paper from Wouter Bulten... 
        if len(matches) > 1:
            comment = 'multiple doi matches, took the first one with most citations'
            match = matches[matches['ss_citations'] == matches['ss_citations'].max()].iloc[0]
        ss_id = match['ss_id']
        doi_match_data.append([bibkey, ss_id, comment])

df_doi_match_data = pd.DataFrame(doi_match_data, columns = match_keys_columns)
df_doi_match_data.to_csv('script_data/temp_doi_match_data.csv', index=False)
print('DONE')
print(len(df_doi_match_data['comment'].notna()))
print(len(df_doi_match_data))

[MATCHING ON DOIS]
DONE
323
323


In [59]:
matches

,staff_id,ss_id,title,doi,ss_citations
9,8038506,c661857719657aac571718e3d48cb66b8fc1e941,Prostate158 - An expert-annotated 3T MRI datas...,10.1016/j.compbiomed.2022.105817,4
743,145959882,c661857719657aac571718e3d48cb66b8fc1e941,Prostate158 - An expert-annotated 3T MRI datas...,10.1016/j.compbiomed.2022.105817,4


In [64]:
import time

9.36703872680664

In [72]:
start = time.time()
print('[MATCHING ON TITLES]')
title_match_data = []
for idx_ss, row_ss in df_all_staff_id_ss_data.iterrows():
    if idx_ss % 50 == 0:
        print(f'[{idx_ss}/{len(df_all_staff_id_ss_data)}]')
    
    ss_id = row_ss['ss_id']
    if ss_id in df_doi_match_data['ss_id']:
        continue
 
    ss_title = row_ss['title']
    if ss_title == None:
        continue
    
    title_match_ratios = df_bib_data_with_titles['title'].apply(lambda x: SequenceMatcher(
        a=ss_title.lower(), 
        b=x.lower().replace('{', '').replace('}', '')).ratio())

    max_ratio = title_match_ratios.max()
    max_bibkey = df_bib_data_with_titles[title_match_ratios==max_ratio]['bibkey'].iloc[0]
    max_bib_title = df_bib_data_with_titles[title_match_ratios==max_ratio]['title'].iloc[0]
    max_bib_title = max_bib_title.replace('{', '').replace('}', '')
    
    up80_bibkeys = None
    up80_bib_titles = None
    up80_ratios = None
    if sum(title_match_ratios>0.8) > 1:
        
        up80_bibkeys = list(df_bib_data_with_titles[title_match_ratios>0.8]['bibkey'])
        up80_bib_titles = list(df_bib_data_with_titles[title_match_ratios>0.8]['title'])
        up80_bib_titles = [up80_bib_title.replace('{', '').replace('}', '') for up80_bib_title in up80_bib_titles]
        up80_ratios = list(title_match_ratios[title_match_ratios>0.8])
    
    title_match_data.append([ss_id, ss_title, 
                             max_bibkey, max_bib_title, max_ratio, 
                             up80_bibkeys, up80_bib_titles, up80_ratios])

title_matches_columns = ['ss_id', 'ss_title', 
                         'max_bibkey' , 'max_bib_title', 'max_ratio',
                         'up80_bibkeys' , 'up80_bib_titles', 'up80_ratios',
                        ]
df_title_match_data = pd.DataFrame(title_match_data, columns=title_matches_columns)
df_title_match_data.to_csv('script_data/temp_title_match_data.csv', index=False)
print('DONE')
end = time.time()
sec = end-start
print(sec, 'sec')
print(sec/60, 'min')

[MATCHING ON TITLES]
[0/1108]
[50/1108]
[100/1108]
[150/1108]
[200/1108]
[250/1108]
[300/1108]
[350/1108]
[400/1108]
[450/1108]
[500/1108]
[550/1108]
[600/1108]
[650/1108]
[700/1108]
[750/1108]
[800/1108]
[850/1108]
[900/1108]
[950/1108]
[1000/1108]
[1050/1108]
[1100/1108]
DONE
659.5768692493439 sec
10.992947820822398 min


In [52]:
[ss_id, ss_title, max_bibkey, max_bib_title, max_ratio, up80_bibkeys, up80_bib_titles, up80_ratios]

['979a9f247700d00ff2c3f0612d5eb001379f93c8',
 'The Medical Segmentation Decathlon',
 'Anto21',
 'The Medical Segmentation Decathlon',
 1.0,
 ['Anto21', 'Anto22'],
 ['The Medical Segmentation Decathlon', 'The Medical Segmentation Decathlon'],
 [1.0, 1.0]]

In [48]:
df_bib_data.iloc[1086]

bibkey                                      Anto21
title           The Medical Segmentation Decathlon
doi                                           None
gs_citations                                  None
Name: 1086, dtype: object

In [49]:
df_bib_data.iloc[1087]

bibkey                                           Anto22
title           The {Medical} {Segmentation} {Decathlon
doi                          10.1038/s41467-022-30695-9
gs_citations                                       None
Name: 1087, dtype: object

In [24]:
max_ratio = title_match_ratios.max()
max_bibkey = df_bib_data_with_titles[title_match_ratios==max_ratio]['bibkey'].iloc[0]
max_bib_title = df_bib_data_with_titles[title_match_ratios==max_ratio]['title'].iloc[0]
max_bib_title = max_bib_title.replace('{', '').replace('}', '')


up80_bibkeys = None
un80_bib_titles = None


In [25]:
[ss_id, bibkey, max_ratio, bib_title, ss_title]

['979a9f247700d00ff2c3f0612d5eb001379f93c8',
 'Ginn20',
 1.0,
 'The Medical Segmentation Decathlon',
 'The Medical Segmentation Decathlon']

In [17]:
df_bib_data_with_titles[title_match_ratios==max_ratio]['bibkey']

NameError: name 'title_match_ratios' is not defined

In [194]:
bib_title

'Pulmonary nodule detection in CT images: false positive reduction using multi-view convolutional networks'

In [196]:
SequenceMatcher(a='AAaa', b="aaaa").ratio()

0.5

In [199]:
df_title_matches[df_title_matches['max_ratio']<=0.8].sort_values('max_ratio')

,bibkey,ss_id,max_ratio,bib_title,ss_title
21,Putz19,f574d9f2b018d707ff0c2dbcd250fcd4ee2856ae,0.485714,i-RIM applied to the fastMRI challenge,Learn2Reg - The Challenge (2021)
74,Loog06,cf46e880665be3dd1b2a81cc1be53f1ea9d64de1,0.500000,Bony Structure Suppression in Chest Radiographs,Context Encoding Chest X-rays
4,Marc22,2d49fb82fd715a213e2149f9f08dd2fb4749b235,0.500000,Empowering digital pathology applications thro...,Challenges in digital medicine applications in...
79,Maie19,f16773597d32d0c6d2ab1b66465e26dde06eadac,0.502674,BIAS: Transparent reporting of biomedical imag...,Is the winner really the best? A critical anal...
5,Kall11,2f2182f8e55be5a85c1316cd1b181cd5c85c106c,0.503497,Automatic breast density segmentation based on...,Metrics reloaded: Pitfalls and recommendations...
37,Heri22,ee17bc918583166ad08ff307be5f77cb130486ea,0.529412,Deep-Learning-Based Image Registration and Tum...,Constraining Volume Change in Learned Image Re...
0,Litj17,1898998f27ce0750a42f8f3ca2ed7e292f05aee2,0.529915,A Survey on Deep Learning in Medical Image Ana...,Understanding metric-related pitfalls in image...
33,Kabu09,888c62de7448e14302c3c2ed7dc062829ea279d7,0.531915,Evaluation of 4D-CT Lung Registration,Learn2Reg Challenge: CT Lung Registration - Tr...
7,Less18a,5a09637587e694a03f68a4ee1046d31aa97fd0c0,0.533333,Iterative fully convolutional neural networks ...,Structure and position-aware graph neural netw...
34,Putz19,99cf88b6359e0a5354c809fdb992bf82fdffc929,0.539683,i-RIM applied to the fastMRI challenge,Learn2Reg - The Challenge


In [201]:
df_title_matches[df_title_matches['max_ratio']>0.8]

,bibkey,ss_id,max_ratio,bib_title,ss_title
6,Vent21,41f386d38567e38132525cad9bdc7da1ad6e8f1c,1.000000,Automated COVID-19 Grading with Convolutional ...,Automated COVID-19 Grading With Convolutional ...
8,Band23,bc6b483f9b1fa630fa4b43158b13716f1ca7497b,0.878613,Continual learning strategies for cancer-indep...,Domain adaptation strategies for cancer-indepe...
9,Adam22b,c661857719657aac571718e3d48cb66b8fc1e941,0.994595,Prostate158 - An expert-annotated 3T MRI datas...,Prostate158 - An expert-annotated 3T MRI datas...
10,Soga22,1d5f65d8f721089fca1e7fac8d1ef214f12e2c23,1.000000,Automated estimation of total lung volume usin...,Automated estimation of total lung volume usin...
11,Xie21,29595323a6789fec907a39c4de1ad56b75c74630,1.000000,Deep Clustering Activation Maps for Emphysema ...,Deep Clustering Activation Maps for Emphysema ...
...,...,...,...,...,...
94,Hump17,b704867c644d95b8ddc980b19b3805181b7a28ab,1.000000,Organ detection in thorax abdomen CT using mul...,Organ detection in thorax abdomen CT using mul...
95,Pate17a,be15a442d2cf19f1bbbd779273055a2de727380c,1.000000,Automatic Cerebrospinal Fluid Segmentation in ...,Automatic cerebrospinal fluid segmentation in ...
97,Char16c,d884880febbc8e6c5f86c1040c009174632569c4,1.000000,Improving Airway Segmentation in Computed Tomo...,Improving airway segmentation in computed tomo...
98,Lief17,e4010e494f6a7f0ea93b4cdb257aa1e51611609a,1.000000,Fovea Detection in Optical Coherence Tomograph...,Fovea detection in optical coherence tomograph...


In [162]:
df_bib_data[title_match_ratios==max_ratio]

C:\Users\joeyspronck\AppData\Local\Temp\ipykernel_41512\2482172138.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_bib_data[title_match_ratios==max_ratio]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [152]:
row_ss

bibkey                                                     Abas05
title           Analysis of regularity in the {EEG} background...
doi                                  10.1016/j.clinph.2005.04.001
gs_citations                                                  233
Name: 1065, dtype: object

In [150]:
df_bib_data[df_bib_data['title'].notna()]

,bibkey,title,doi,gs_citations
1065,Abas05,Analysis of regularity in the {EEG} background...,10.1016/j.clinph.2005.04.001,233
1066,Genu22,Imaging tumor-infiltrating CD8 (+) T-cells in ...,None,None
1067,Abas05a,EEG} background activity analysis in {A}lzheim...,None,3
1068,Abel19,"Computational pathology definitions, best prac...",10.1002/path.5331,24
1069,Abra08a,Evaluation of a system for automatic detection...,10.2337/dc07-1312,287
...,...,...,...,...
2778,Sadr23,Deep Learning for Detection of Periapical Radi...,10.1016/j.joen.2022.12.007,None
2779,Anto23,Retrospective validation of nodule management ...,None,None
2780,Sido23,Multi-source data approach for personalized ou...,10.1007/s10654-023-00975-9,None
2781,Graa23,MRI image features with an evident relation to...,10.1007/s00586-023-07602-x,None


In [146]:
len(df_match_keys)

37

In [144]:
df_title_matches

,bibkey,ss_id,max_ratio,bib_title,ss_title
0,Abas05,88 5f74400c97e716a85f20871b4b477dbca9680664...,0.338129,Analysis of regularity in the {EEG} background...,88 Comparison of the effects of model-based...
1,Genu22,8 bc6b483f9b1fa630fa4b43158b13716f1ca7497b ...,0.360190,Imaging tumor-infiltrating CD8 (+) T-cells in ...,8 Domain adaptation strategies for cancer-i...
2,Abas05a,28 3db60e34dad56e3949508d8dc1324d5331f9a2c6...,0.387097,EEG} background activity analysis in {A}lzheim...,28 Image-level detection of arterial occlus...
3,Abel19,5 2f2182f8e55be5a85c1316cd1b181cd5c85c106c ...,0.362832,"Computational pathology definitions, best prac...",5 Metrics reloaded: Pitfalls and recommenda...
4,Abra08a,57 d0760faf253e27b6c105d76d06acc4c6ab3674c7...,0.602151,Evaluation of a system for automatic detection...,57 Evaluation of a deep learning system for...
...,...,...,...,...,...
1712,Sadr23,82 102a580a89b7e9d5a6d8e79864db183d6b5fb6ad...,0.405530,Deep Learning for Detection of Periapical Radi...,82 Transfer Learning for Domain Adaptation ...
1713,Anto23,24 2cbfeefc3025a11beced3681bd0b358d44d4c0a8...,0.437870,Retrospective validation of nodule management ...,24 ESR/ERS statement paper on lung cancer s...
1714,Sido23,89 63c9344c2046a0403941eb8a91d307b9e1190928...,0.442553,Multi-source data approach for personalized ou...,89 MA 14.11 Malignancy Risk Prediction of P...
1715,Graa23,46 4654aa505e5bcdb089d0df202cd7ceabc9d2d41f...,0.386740,MRI image features with an evident relation to...,46 A large annotated medical image dataset ...


In [138]:
# df_match_keys

In [129]:
'Anto22' in df_match_keys['bibkey']

False

In [114]:
df_staff_id_ss_data[doi_matching]['ss_citations'].max()

100

In [120]:
df_staff_id_ss_data[doi_matching][df_staff_id_ss_data[doi_matching]['ss_citations'] == df_staff_id_ss_data[doi_matching]['ss_citations'].max()].iloc[0]

ss_id                    d5b8fba71671814445de80d36a2d563f43eb9d07
title           Epithelium segmentation using deep learning in...
doi                                    10.1038/s41598-018-37257-4
staff_id                                                  8038506
ss_citations                                                  100
Name: 76, dtype: object

In [90]:
df_staff_id_ss_data['doi']

0        10.48550/arXiv.2302.01790
1                             None
2        10.48550/arXiv.2302.01738
3     10.1007/978-3-031-28241-6_18
4       10.1038/s41467-022-30728-3
                  ...             
95              10.1117/12.2254022
96                            None
97     10.1016/j.media.2016.11.001
98              10.1117/12.2254301
99        10.1109/TMI.2016.2536809
Name: doi, Length: 100, dtype: object

In [94]:
df_staff_id_ss_data[doi_matching]['ss_id'].item()

'979a9f247700d00ff2c3f0612d5eb001379f93c8'

In [96]:
row_bib

bibkey                                           Anto22
title           The {Medical} {Segmentation} {Decathlon
doi                          10.1038/s41467-022-30695-9
gs_citations                                       None
Name: 1087, dtype: object

In [71]:
row_ss['title']

'Pulmonary Nodule Detection in CT Images: False Positive Reduction Using Multi-View Convolutional Networks'

'10.1117/12.2611423'

In [78]:
df_staff_id_ss_data['doi'].isin([row_bib['doi']]).any()

False

In [74]:
pd.isin(row_bib['doi'], df_staff_id_ss_data['doi'])

AttributeError: module 'pandas' has no attribute 'isin'

In [ ]:
bib_title = bib_entry.fields['title'].strip('{ }')
ss_title = ss_entry

In [ ]:
all_types = []
for entry in diag_bib_raw:
    all_types.append(entry.type)
np.unique(all_types, return_counts=True)

In [ ]:
selected_entries = []
for entry in diag_bib_raw:
    if entry.type in ['article']: #['article', 'book', 'conference', 'inbook', 'inproceedings']
        selected_entries.append(entry)
        print(entry.fields)

In [ ]:
[print(key) for key in selected_entries[0].fields.keys()];